In [2]:
import os 
import ctranslate2


# full_model_path = 'subirmansukhani/llama-2-7b-miniguanaco'
full_model_path = '/mnt/artifacts/llama2/final_merged_checkpoint/'
# Copy the file over from this path to /mnt/data/ or wherever your Domino datasets is if you intend to use this model in an app
ct2_path ='/mnt/artifacts/llama2/llama2-ct/'
quantization ='int8'

os.system(f"sudo ct2-transformers-converter --model {full_model_path} --output_dir {ct2_path} --quantization {quantization}  --force")

print(" Model Converted successfully")

Loading checkpoint shards: 100%|██████████| 2/2 [01:39<00:00, 49.94s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


 Model Converted successfully


In [ ]:
import torch 
import transformers

model_path = '/mnt/data/llama2-ct'
model_device = 'cuda' if torch.cuda.is_available() else 'cpu'

# load the ctranslate model
generator = ctranslate2.Generator(model_path, device=model_device)
tokenizer = transformers.AutoTokenizer.from_pretrained('subirmansukhani/llama-2-7b-miniguanaco')

In [ ]:
import time

prompt = "Complete the following Python function that computes the factorial of a number: \ndef factorial(n):"
text = f"<s>[INST] {prompt} [/INST]"
pct_new_tokens = 1.2

tokens = tokenizer.convert_ids_to_tokens(tokenizer.encode(text))
input_length = len(tokens)
new_tokens = round(pct_new_tokens*input_length)
tokens_per_sec = 0
start_time = time.time()
results = generator.generate_batch([tokens], sampling_topk=10, max_length=new_tokens, include_prompt_in_result=False)
end_time = time.time()
output_text = tokenizer.decode(results[0].sequences_ids[0])
tokens_per_sec = round(new_tokens / (end_time - start_time),3)
tokens_per_sec

In [ ]:
output_text